In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

In [2]:
conn = pymongo.MongoClient()
cine = conn.cine
collection_cine2 = cine.collection_cine2

In [3]:
collection_cine2.find_one()

### index
- 검색 속도를 향상시키기 위해서 사용하는 데이터 구조
- index 를 사용하지 않으면 collection 에 있는 document 를 
- 하나씩 조회하는 방식으로 검색함

### 기본 index
- MongDB 의 collection 은 기본적으로 _id key 기반으로 index 가 생성됨
  -  Single Field index (단일 필드 index)
  -  Compound Field index (복합 필드 index)

In [4]:
'''
단일 필드 index : 하나의 field 로 index 를 생성함
 <-- 컬렉션이름.create_index(key(field))
'''
collection_cine2.create_index("배우 이름")

'배우 이름_1'

In [5]:
'''
index 의 기본 정보 조회하기
  <-- 컬렉션이름.index_information()
'''
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'}}

- key("필드이름", direction)
  -  direction
    -  pymongo.ASCENDING = 1 : 오름차순 (default)
    -  pymongo.DESCENDING = -1 : 내림차순
    -  pymongo.TEXT = "text" : text 검색을 쉽게 할 수 있도록 함

### index 삭제하기
- 전체 index 삭제
- 컬렉션이름.drop_indexes()
-  ㄴ 기본 index 이외의 모든 index 가 삭제됨

In [6]:
collection_cine2.drop_indexes()

In [7]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'}}

- 특정 index 삭제하기
- 컬렉션이름.drop_index([('필드명',1)])

In [8]:
collection_cine2.create_index("배우 이름")
collection_cine2.create_index("랭킹")
collection_cine2.create_index("흥행지수")

'흥행지수_1'

In [9]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)], 'ns': 'cine.collection_cine2'},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)], 'ns': 'cine.collection_cine2'}}

In [10]:
'''
"직업" key(field) 로 index 를 생성하면서
DESCENDING 으로 설정하기
'''
collection_cine2.create_index([("직업", pymongo.DESCENDING)])

'직업_-1'

In [11]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)], 'ns': 'cine.collection_cine2'},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)], 'ns': 'cine.collection_cine2'},
 '직업_-1': {'v': 2, 'key': [('직업', -1)], 'ns': 'cine.collection_cine2'}}

In [12]:
collection_cine2.create_index([("출연영화", "text")])

'출연영화_text'

In [13]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'},
 '랭킹_1': {'v': 2, 'key': [('랭킹', 1)], 'ns': 'cine.collection_cine2'},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)], 'ns': 'cine.collection_cine2'},
 '직업_-1': {'v': 2, 'key': [('직업', -1)], 'ns': 'cine.collection_cine2'},
 '출연영화_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('출연영화', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [14]:
collection_cine2.drop_indexes()

In [15]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'}}

In [16]:
collection_cine2.create_index("배우 이름")

'배우 이름_1'

In [17]:
documents = \
  collection_cine2.find({"배우 이름":"김윤석"}, {"배우 이름":1, "출연영화":1, "_id":0})

for document in documents:
    print(document)

In [18]:
import pprint

documents = \
  collection_cine2.find({"배우 이름":"김윤석"})

for document in documents:
    pprint.pprint(document)

In [19]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'}}

In [20]:
collection_cine2.create_index([("직업", "text")])

'직업_text'

In [21]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'},
 '직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [22]:
documents = collection_cine2.find({"$text":{"$search":"가수"}})

for document in documents:
    pprint.pprint(document)

In [23]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_1': {'v': 2, 'key': [('배우 이름', 1)], 'ns': 'cine.collection_cine2'},
 '직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [24]:
collection_cine2.drop_index([("배우 이름", 1)])

In [25]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [26]:
collection_cine2.drop_index([("직업", "text")])

In [27]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'}}

In [28]:
collection_cine2.create_index([("배우 이름", "text")])


'배우 이름_text'

In [29]:
collection_cine2.create_index([("출연영화", 1)])

'출연영화_1'

In [30]:
collection_cine2.drop_index([("출연영화", 1)])

In [31]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('배우 이름', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [32]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '배우 이름_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('배우 이름', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [33]:
documents = collection_cine2.find({"$text" : {"$search":"박해일"}})
for document in documents:
    pprint.pprint(document)

In [34]:
documents = collection_cine2.find({"$text" : {"$search":"김윤석"}})
for document in documents:
    pprint.pprint(document)

-  Compound Field index (복합 필드 index)
  - 최대 31 개의 필드로 index 를 생성할 수 있음
  - 여러 개의 필드를 하나의 index 로 지정함

In [35]:
collection_cine2.drop_indexes()

In [36]:
'''
복합 필드 인덱스 생성하기
'''
collection_cine2.create_index(
                                 [("출연영화",pymongo.TEXT), ("직업", pymongo.TEXT),("흥행지수", pymongo.DESCENDING)]                                 
                             )

'출연영화_text_직업_text_흥행지수_-1'

In [37]:
collection_cine2.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_cine2'},
 '출연영화_text_직업_text_흥행지수_-1': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1), ('흥행지수', -1)],
  'ns': 'cine.collection_cine2',
  'weights': SON([('직업', 1), ('출연영화', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

### 복합 필드 인덱스로 부분문자열 검색(조회)하기

In [38]:
for document in collection_cine2.find({"$text":{"$search":"가수"}}):
    pprint.pprint(document)

In [39]:
for document in collection_cine2.find({"$text":{"$search":"모가디슈"}}):
    pprint.pprint(document)

### 정규표현식(Regular Expression) 을 사용해서 부분문자열 검색하기

In [40]:
'''
출연영화에 "함께"라는 문자열이 있는 document 조회하기
'''
for document in collection_cine2.find({"출연영화":{"$regex":"함께"}}):
    pprint.pprint(document)

In [41]:
'''
출연영화에 "국가"라는 문자열이 있는 document 조회하기
'''
for document in collection_cine2.find({"출연영화":{"$regex":"국가"}}):
    pprint.pprint(document)

In [42]:
'''
collection_text 라는 컬렉션 생성하기
'''
collection_text = cine.collection_text

In [43]:
doc1 = {"name":"java cafe", "description":"Coffe & Cake", "ranking":1}
doc2 = {"name":"Burger Pot", "description":"Coffe & Burgar", "ranking":2}
doc3 = {"name":"Coffee Shop", "description":"Only Coffee", "ranking":3}
doc4 = {"name":"Clothes Shop", "description":"Discount Clothes", "ranking":4}
doc5 = {"name":"Shopping Site", "description":"Shopping & Rest", "ranking":5}

In [44]:
collection_text.insert_many(
    [doc1, doc2, doc3, doc4, doc5]
)

In [45]:
'''
regex
  .  문자 하나
  +  하나 이상
  *  없거나 또는 그 이상
'''

for document in collection_text.find({"name" : {"$regex" : "Co.+"}}, {"_id":0}):
    print(document)

{'name': 'Coffee Shop', 'description': 'Only Coffee', 'ranking': 3}


In [46]:
for document in collection_text.find({"name": {"$regex":"Co.*"}}, {"_id":0}):
    print(document)

{'name': 'Coffee Shop', 'description': 'Only Coffee', 'ranking': 3}


In [47]:
collection_text.insert_one(
    {"name":"Co", "description":"cafe", "ranking":6}
)

In [48]:
for document in collection_text.find({"name": {"$regex":"Co.*"}}, {"_id":0}):
    print(document)

{'name': 'Coffee Shop', 'description': 'Only Coffee', 'ranking': 3}
{'name': 'Co', 'description': 'cafe', 'ranking': 6}


In [49]:
for document in collection_text.find({"name": {"$regex":"Co.+"}}, {"_id":0}):
    print(document)

{'name': 'Coffee Shop', 'description': 'Only Coffee', 'ranking': 3}


In [50]:
'''
name 필드에 text 속성을 지정해서 index 생성하기
'''
collection_text.create_index([("name", "text")])

'name_text'

In [51]:
collection_text.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine.collection_text'},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine.collection_text',
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [52]:
for document in collection_text.find({"$text":{"$search":"Coffee"}}, {"_id":0}):
    pprint.pprint(document)

{'description': 'Only Coffee', 'name': 'Coffee Shop', 'ranking': 3}


In [53]:
documents = \
  collection_text.find({"$text":{"$search":"Java coffee shop"}}, {"_id":0})

for document in documents:
    pprint.pprint(document)

{'description': 'Only Coffee', 'name': 'Coffee Shop', 'ranking': 3}
{'description': 'Coffe & Cake', 'name': 'java cafe', 'ranking': 1}
{'description': 'Shopping & Rest', 'name': 'Shopping Site', 'ranking': 5}
{'description': 'Discount Clothes', 'name': 'Clothes Shop', 'ranking': 4}


In [54]:
''' 
 "검색할문자열" : "" 속에 지정한 문자열과 일치하는 내용만 조회함
'''
documents = \
  collection_text.find({"$text":{"$search": "\"Java coffee shop\""}}, {"_id":0})

for document in documents:
    print(document)

In [55]:
documents = \
  collection_text.find({"$text":{"$search": "\"coffee shop\""}}, {"_id":0})

for document in documents:
    print(document)

{'name': 'Coffee Shop', 'description': 'Only Coffee', 'ranking': 3}


In [56]:
documents = \
  collection_text.find({"$text":{"$search": "\"burger pot\""}}, {"_id":0})

for document in documents:
    print(document)

{'name': 'Burger Pot', 'description': 'Coffe & Burgar', 'ranking': 2}


In [64]:
'''
대소문자 구분해서 일치하는 내용 조회하기

  "$caseSensitive":True 
  
  라고 설정함
'''

documents = \
  collection_text.find({"$text":{"$search":"\"coffee shop\"", "$caseSensitive":True}}, {"_id":0})

for document in documents:
    print(document)

In [63]:
for document in collection_cine2.find({}, {"학교":1, "배우 이름":1, "_id":0}):
    pprint.pprint(document)

{'배우 이름': '박해일', '학교': '남서울대학교 영어학 (중퇴)'}
{'배우 이름': '박지환'}
{'배우 이름': '손석구'}
{'배우 이름': '현빈', '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중'}
{'배우 이름': '최귀화'}
{'배우 이름': '허동원'}
{'배우 이름': '변요한'}
{'배우 이름': '정우성'}
{'배우 이름': '유해진', '학교': '서울예술대학 연극과'}
{'배우 이름': '하준'}
{'배우 이름': '김성균'}
{'배우 이름': '정재광'}
{'배우 이름': '음문석'}
{'배우 이름': '안성기', '학교': '한국외국어대학교 베트남어학 학사'}
{'배우 이름': '임윤아'}
{'배우 이름': '이정재'}
{'배우 이름': '남문철'}
{'배우 이름': '손현주', '학교': '중앙대학교 연극영화'}
{'배우 이름': '송강호', '학교': '부산경상대학 방송연예'}
{'배우 이름': '박지영'}
{'배우 이름': '전혜진', '학교': '상명대학교 영화과'}
{'배우 이름': '김윤석', '학교': '동의대학교 학사'}
{'배우 이름': '고경표'}
{'배우 이름': '김성규'}
{'배우 이름': '조우진'}
{'배우 이름': '이주원'}
{'배우 이름': '조인성', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)'}
{'배우 이름': '허성태'}
{'배우 이름': '김소진'}
{'배우 이름': '신시아'}
{'배우 이름': '이규원'}
{'배우 이름': '허준호', '학교': '서울예술대학 연극과'}
{'배우 이름': '박은빈', '학교': '영파여자고등학교'}
{'배우 이름': '진선규'}
{'배우 이름': '김찬형'}
{'배우 이름': '정만식'}
{'배우 이름': '김향기'}
{'배우 이름': '이광수'}
{'배우 이름': '서은수'}
{'배우 이름': '강하늘', '학교': '중앙대학교 연극학과'}
{'배우 이름': '고윤정', '학교': '서울여

In [61]:
'''
서울예술대학 출신을 흥행지수 내림차순으로 10 명 조회하기
(정규표현식 사용하기)
출력필드 : 배우 이름, 학교, 흥행지수
'''
documents = collection_cine2.find({"학교":{"$regex":"서울예술대학"}},\
                                  {"배우 이름":1, "학교":1, "흥행지수":1, "_id":0})\
                            .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)

{'배우 이름': '유해진', '학교': '서울예술대학 연극과', '흥행지수': 16882}
{'배우 이름': '허준호', '학교': '서울예술대학 연극과', '흥행지수': 9355}
{'배우 이름': '황정민', '학교': '서울예술대학 연극과 졸업', '흥행지수': 6618}
{'배우 이름': '전도연', '학교': '서울예술대학 방송연예', '흥행지수': 6207}
{'배우 이름': '정우', '학교': '서울예술대학 영화과', '흥행지수': 2624}
{'배우 이름': '박희순', '학교': '서울예술대학 연극과', '흥행지수': 2026}
{'배우 이름': '김희원', '학교': '서울예술대학 연극과', '흥행지수': 1748}
{'배우 이름': '류승룡', '학교': '서울예술대학 연극', '흥행지수': 1725}


In [62]:
documents = collection_cine2.find({"학교":{"$regex":"중앙대학교"}},\
                                  {"배우 이름":1, "학교":1, "흥행지수":1, "_id":0})\
                            .sort("흥행지수", pymongo.DESCENDING).limit(10)

for document in documents:
    pprint.pprint(document)

{'배우 이름': '현빈', '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중', '흥행지수': 21073}
{'배우 이름': '손현주', '학교': '중앙대학교 연극영화', '흥행지수': 13418}
{'배우 이름': '강하늘', '학교': '중앙대학교 연극학과', '흥행지수': 7775}
{'배우 이름': '이정현', '학교': '중앙대학교 연극영화', '흥행지수': 7277}
{'배우 이름': '박용우', '학교': '중앙대학교 연극영화과', '흥행지수': 6770}
{'배우 이름': '김강우', '학교': '중앙대학교 연극영화학', '흥행지수': 2374}
{'배우 이름': '권율', '학교': '중앙대학교 연극학과', '흥행지수': 1774}
